### European Central Bank API

This notebook retrieves monthly data on the yield of comparable 10-year government bonds of Eurozone countries from the [ECB API](https://sdw-wsrest.ecb.europa.eu/web/generator/index.html), along with the one year high and low values.

In [1]:
import requests
import pandas as pd

# API in JSON data format returns series index as strings
# tiny function below used to convert to ints during sort
def asint(s):
    return int(s)

# Short script to write labels to txt files for charts
def write_txt(filename, filetext):
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

In [2]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
clist = ['DE', 'FR', 'IT', 'ES', 'PT', 'GR', 'NL', 'BE', 'AT', 'IE']

d = {}    # Empty dictionary used to store important values
for country in clist:
    key = 'IRS/M.{}.L.L40.CI.0000.EUR.N.Z?startPeriod=2012-01-01'.format(country)
    response = requests.get('{}{}'.format(url,key), headers=headers).json()
    # API returns data series in one dictionary and dates in another
    series = response['dataSets'][0]['series'][u'0:0:0:0:0:0:0:0:0']['observations']
    slist = [series[k][0] for k in sorted(series, key=asint)]
    dates = response['structure']['dimensions']['observation'][0]['values']
    dlist = [i['start'][:10] for i in dates]
    area = response['structure']['dimensions']['series'][1]['values'][0]['name']
    # Convert two lists into pandas dataframe
    df = pd.DataFrame({'date': dlist, area: slist}).set_index('date')
    d[country] = {}     # Create key in dictionary for each country
    d[country]['mark'] = df[area][-1]
    d[country]['A'] = area
    d[country]['min'] = df[area][-60:].min()  #Most recent year only
    d[country]['max'] = df[area][-60:].max()
recent_date = pd.to_datetime(df.index[-1]).strftime('%B %Y')  # Latest date as text

In [3]:
# Convert dictionary to pandas dataframe
df = pd.DataFrame(d).transpose().sort_values('mark')
# For plot, separate positive and negative minima
df['neg'] = df['min'].clip_upper(0)
df.loc[:,'min'] = df['min'].clip_lower(0)
df['max'] = df['max'].subtract(df['min'], fill_value=0)
# Reorder dataframe
df = df[['A', 'mark', 'max', 'min', 'neg']]
# Write to csv and write date to text file
df.to_csv('int.csv', index=False)
write_txt('int.txt', recent_date)

In [4]:
recent_date

'October 2017'